In [ ]:
from google.colab import drive
from nltk.corpus import stopwords
import nltk
from collections import defaultdict
import re
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from google.colab import drive
import heapq
import json
import gc
import csv
import sys
porter = PorterStemmer()

In [ ]:
# This code will connect our notebook to the google drive and we can easily fetch data from there
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# From here we are basically retrieving the stop words from the nltk library which are printed here
nltk.download('stopwords')
stop_words = stopwords.words('english')
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# This function will help us to perform regular expression operation and filter more data.
def remove_symbols(line):
    return re.sub('[^A-Za-z0-9\s]+', '', line).lower()


In [ ]:
# Now as we are going to read the CSV file so we will need higher field limit then the default so we can extend that using below code
csv.field_size_limit(sys.maxsize)

131072

In [ ]:
# Through below code we are reaching to the directory in which we have the gutenberd_data.csv. This is for my drive may vary for others
%cd /content/drive/MyDrive/Information\ Retrieval\ Material
%ls

/content/drive/MyDrive/Information Retrieval Material
gutenberg_data.csv  Outputs/


In [ ]:
BLOCK_SIZE = 100000

In [ ]:
# BSBI ROUTINE WHICH HELPS US MAKING EFFICIENT POSTING LISTS
def bsbi():
  freq_dict = defaultdict(set)
  with open('gutenberg_data.csv') as f:
    # Using the below line next(f) we are skipping the new column header
    next(f)
    csv_file = csv.reader(f)
    total_files = 0
    i = 0
    current_block = 0
    for line in csv_file:
      # we are extracting title, author,link, id, bookshelf, text from a particular line or entry in the CSV
      title, author, link, id, bookshelf, text = line
      i += 1

      # Here we will process the words that comes under the text
      for word in text.split():
        # here we are using the routine "remove_symbols" made above that will remove some symbols from the word.
        word = remove_symbols(word)

        # if the word exist and it is not in stop_words then we will stem the word
        if word and word not in stop_words:
          word = porter.stem(word)
          if word not in freq_dict:
            # if word is not added before, we will increase block size too
            current_block += 1
          """
          note: __contains__ will just check if doc is already there.
          It's not exactly needed as we are using set, but we are checking it to manage block size
          also searching in sets is faster as they are hashed (while lists are not hashed)
          """
          if not freq_dict[word].__contains__(id):
            freq_dict[word].add(id)
            current_block += 1
        if current_block >= BLOCK_SIZE:
          # LETS DO THE WRITE OPERATION
          # We will now sort the list before writing according to the word_id
          sorted_list = sorted(freq_dict.items(), key=lambda _: _[0]) # sorting by word_id

          with open(f'./Outputs/op{total_files}.txt', 'w') as  f:
            # json.dump(freq_dict, f, cls=SetEncoder)
            for word_id, docs in sorted_list:
              f.write(word_id)
              for doc_id in docs:
                f.write(f' {doc_id}')
              f.write('\n')
          current_block = 0
          freq_dict.clear()
          total_files += 1
          print(i, ' rows done')
          # if total_files == 2:
          #   return

    sorted_list = sorted(freq_dict.items(), key=lambda _: _[0]) # sorting by word_id
    # this is for last values
    # TODO:: DO IT BY FN SO NO REPEATATION OF CODE
    if len(sorted_list) > 0:
      with open(f'./Outputs/op{total_files}.txt', 'w') as  f:
        # json.dump(freq_dict, f, cls=SetEncoder)
        for word_id, docs in sorted_list:
          f.write(word_id)
          for doc_id in docs:
            f.write(f' {doc_id}')
          f.write('\n')
      current_block = 0
      freq_dict.clear()
      total_files += 1
bsbi()

15  rows done
23  rows done
41  rows done
54  rows done
67  rows done
81  rows done
96  rows done
116  rows done
130  rows done
146  rows done
157  rows done
181  rows done
207  rows done
220  rows done
254  rows done
271  rows done
280  rows done
291  rows done
302  rows done
332  rows done
353  rows done
371  rows done
387  rows done
400  rows done
444  rows done
456  rows done
468  rows done
476  rows done
488  rows done
499  rows done
506  rows done
509  rows done
518  rows done
526  rows done
533  rows done
541  rows done
548  rows done
565  rows done
590  rows done
606  rows done
616  rows done
627  rows done
634  rows done
641  rows done
649  rows done
660  rows done
678  rows done
685  rows done
693  rows done
703  rows done
718  rows done
732  rows done
740  rows done
749  rows done
757  rows done
762  rows done
772  rows done
779  rows done
788  rows done
796  rows done
804  rows done
817  rows done
828  rows done
839  rows done
848  rows done
857  rows done
866  rows done
87

KeyboardInterrupt: ignored

In [ ]:
# We had to do it on our local system so we ran this bsbi algo only upto 114 output files and we will be doing the rest on HADOOP
file_names = [f'./Outputs/op{i}.txt' for i in range(114)]
file_pointers = [open(i) for i in file_names]
print(file_pointers)

In [ ]:
"""
We will be merging the posting lists using the heap in python which is much more efficient. Link Below
https://stackoverflow.com/questions/1001569/python-class-to-merge-sorted-files-how-can-this-be-improved
"""

# here we are using yeild so we are JUST READING ONE LINE at a time
def decorated_file(f, key):
  for line in f:
    yield (key(line), line)

files = map(open, file_names)
outfile = open('./Output_2_Merged/merged.txt', 'w')

def key_fn(line):
    return line.split(' ', 2)[0] # returning word_id

"""
This is how we are merging the files using the heap in python

Example:
Assume Two files as shown below:

File 1:

Amaze 1 2 3
Wow 3 4 5

and

File 2:

Amaze 6 7 8
Wow 8 9 10

Output would be
Amaze 1 2 3 6 7 8
Wow 3 4 5 8 9 10
"""

prev = ''
for line in heapq.merge(*[decorated_file(f, key_fn) for f in files]):
  # to understand this, you can do this over small number of sorted files and try to print line
  if prev != line[0]:
    # if we have new word, make sure to add new line at first
    outfile.write(f'\n{line[1].strip()}')
    prev = line[0]
  # if we have same word yet, put a space and add other ids
  else:
    # line[1][len(line[0]):] => We are removing the word_id string and then writing the line
    outfile.write(f' {line[1][len(line[0]):].strip()}')
for i in file_pointers:
  i.close()
outfile.close()